In [104]:
# загружаем библиотеки
import pandas as pd
import scipy.sparse as sparse
import numpy as np
import random
import implicit
from sklearn import preprocessing

In [105]:
df = pd.read_parquet('processed_train_full.parquet', engine='pyarrow')

In [106]:
df.shape

(12224256, 6)

In [107]:
df['user_interaction_level'] = df['event_type'].map({'show_vacancy': 1.0,
                                     'preview_click_vacancy': 2.0,
                                     'click_favorite': 3.0,
                                     'preview_click_favorite': 4.0,
                                     'click_response': 5.0,
                                     'click_contacts': 6.0,
                                     'preview_click_response': 7.0,
                                     'preview_click_contacts': 8.0,
                                     'click_phone': 9.0,
                                     'preview_click_phone': 10.0})

In [108]:
df

,event_date,event_timestamp,vacancy_id_,cookie_id,user_id,event_type,user_interaction_level
0,2022-08-01,1659323026,129850,97990f1a021d4be19aa3f955b7eacab4,951f53de61764ea0b51317200a0dbbfc,show_vacancy,1.0
1,2022-08-01,1659377255,108347,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy,1.0
2,2022-08-01,1659376695,109069,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy,1.0
3,2022-08-01,1659376722,171425,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy,1.0
4,2022-08-01,1659374929,252384,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy,1.0
...,...,...,...,...,...,...,...
12224251,2022-08-11,1660246479,162851,0d3c58c6864546c689fa0997de484b30,18124998fd644e80b667a1d84bf63846,preview_click_response,7.0
12224252,2022-08-11,1660246106,136818,ef559274a81e4ff4b92aa3e9f6805886,5c71c1f1b7cf492ba05871c400a05250,preview_click_response,7.0
12224253,2022-08-11,1660250139,143721,c4453128ae584dca8f2a8d20a7c74263,33c968125459420e89ffbc8cffece317,preview_click_response,7.0
12224254,2022-08-11,1660250318,154423,c4453128ae584dca8f2a8d20a7c74263,33c968125459420e89ffbc8cffece317,preview_click_response,7.0


In [109]:
interactions = df.groupby(['cookie_id', 'vacancy_id_'])['user_interaction_level'].unique().apply(sum)

In [110]:
interactions_df = interactions.reset_index()
interactions_df

,cookie_id,vacancy_id_,user_interaction_level
0,0000c4548c3944c08972bbdc1fa4eb85,137659,3.0
1,0000c4548c3944c08972bbdc1fa4eb85,153975,3.0
2,0000c4548c3944c08972bbdc1fa4eb85,174953,6.0
3,0000c4548c3944c08972bbdc1fa4eb85,176171,3.0
4,0000c4548c3944c08972bbdc1fa4eb85,182445,3.0
...,...,...,...
4668584,ffffdb17f45b4032b386d691d52e6c00,107125,9.0
4668585,ffffdb17f45b4032b386d691d52e6c00,107893,10.0
4668586,ffffdb17f45b4032b386d691d52e6c00,150843,10.0
4668587,ffffdb17f45b4032b386d691d52e6c00,160164,10.0


In [111]:
max_event_strength = interactions_df['user_interaction_level'].max()
print(max_event_strength)

55.0


In [113]:
# выбираем случайную вакансию
random_vacancy = random.choice(interactions_df['vacancy_id_'].unique())
print("Vacancy ID:", random_vacancy)
# фильтруем данные по выбранной вакансии
interactions_df[interactions_df['vacancy_id_'] == random_vacancy].sort_values('user_interaction_level', ascending=False).head(10)

Vacancy ID: 244692


,cookie_id,vacancy_id_,user_interaction_level
452130,18befbe2818c4d00a04492f7ee974f1d,244692,7.0
1211662,423d811f787a4e3e831ca0a38defe24b,244692,7.0
4662455,ffa9dfe4489046f59941d6f827aabb0d,244692,7.0
277687,0f34cbf3adb34179b2e7a7fe58fc463f,244692,3.0
911999,31c52e883f544c548a743ed5535e8c0a,244692,3.0
1798256,62470f80677440878f2833953fe99910,244692,3.0
1934066,69e4a09029304f4da1465fb683e41d1a,244692,3.0
2621867,8f064c5e6e5240c2a618743aed6b8b4e,244692,3.0
3012043,a468ee38a0e94f9699c7cea970b39246,244692,3.0
3247861,b19a355b4ad242a7a04cff3155a0d7ae,244692,3.0


In [159]:
from scipy.sparse import csr_matrix
# 
# Создаем словарь для cookie_id
user_id_dict = {}
for i, uid in enumerate(interactions_df['cookie_id'].unique()):
    user_id_dict[uid] = i


# Заменяем значения в interactions_df на соответствующие индексы
interactions_df['user_id'] = interactions_df['cookie_id'].apply(lambda x: user_id_dict[x])


# Создаем разреженную матрицу
sparse_matrix = csr_matrix((interactions_df['user_interaction_level'], (interactions_df['user_id'], interactions_df['vacancy_id_'])))


In [160]:
print(sparse_matrix.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [161]:
sparse_matrix.shape

(330180, 260168)

In [163]:
# Создаем словарь для cookie_id
user_id_dict = {}
for i, uid in enumerate(interactions_df['cookie_id'].unique()):
    user_id_dict[uid] = i


In [164]:
# Смотрим
for cookie_id, user_id in user_id_dict.items():
    print(cookie_id, "->", user_id)


0000c4548c3944c08972bbdc1fa4eb85 -> 0
0000d7508334414ca792c5ff66eb8c14 -> 1
0000e63f423542fe8090575b611cea4d -> 2
00013bf3e58244b48d4dbdf43543c30c -> 3
00015019b0594984afc3fd9fa9557490 -> 4
0001a23eb7fa4b1db1e933484b03e668 -> 5
00021ea9459044b0af294eb5dd7ab8a7 -> 6
00025878f0534a6a96dac2042dfa07b7 -> 7
00028e0b0d474748ac5a83688a4f8896 -> 8
0002cb657088464b99775a69ce83a2ef -> 9
000344ddc0664127b54f529eaf183e30 -> 10
000385da780542d5a2437cc416e4f063 -> 11
00038b5200304d5d96a05a31ca94751a -> 12
0003e3a2e8c84a4ebfd3a58b7327affc -> 13
0003e60e0c684618baf363fcc19bc2a4 -> 14
00040a7c359b436eada01587aab01945 -> 15
0004451a95a04432b6d8dce117bf3c0c -> 16
00044a0333bc4f44bb80ccf9fc678dba -> 17
000478d4b2964d1b8e53ec9c0e050df3 -> 18
0004b0d059984db291d418066b9f1502 -> 19
0004c1c453a348cd993ef076a8fcafe4 -> 20
000537e8c0224eb78b04dd3827b1be91 -> 21
00053e2a426f444f9a71220b392ddd4a -> 22
00055873d53a4eaeb16d615d263497f1 -> 23
00057860633f4969b8a28a4373ed7f89 -> 24
0005e34ddd854ea287eb6d9884661a85 ->

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [242]:
model_user = implicit.als.AlternatingLeastSquares(factors=195, regularization=3.7097309328656386e-05, iterations=111)

In [243]:
# Обучим модель на разряженной матрице
model_user.fit(sparse_matrix)

  0%|          | 0/111 [00:00<?, ?it/s]

In [241]:
# Подбираем гиперпараметры
import optuna
from sklearn.model_selection import train_test_split
import implicit

# Определяем пространство поиска для гиперпараметров
def objective(trial):
    
    factors = trial.suggest_int('factors', 50, 200, step=20)
    regularization = trial.suggest_loguniform('regularization', 1e-5, 1e-3)
    iterations = trial.suggest_int('iterations', 50, 200, step=20)
    
    # Используем гиперпараметры
    model = implicit.als.AlternatingLeastSquares(factors=factors, regularization=regularization, iterations=iterations)
    model.fit(train)
    
    # Оцениваем модель 
    userid = 0
    ids, scores = model.recommend(userid, sparse_matrix[userid], N=5, filter_already_liked_items=True)
    score = sum(scores)
    
    return score

# Делим
train, val = train_test_split(sparse_matrix, test_size=0.2, random_state=42)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

print("Best Parameters: ", study.best_params)
print("Best Score: ", study.best_value)

[I 2023-04-30 08:39:02,463] A new study created in memory with name: no-name-2d875f13-d5b1-4a3a-b3d2-15adc3619b4e
/var/folders/9g/7jkvb2ws38ddzy34llt9htww0000gn/T/ipykernel_28542/3056477116.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  regularization = trial.suggest_loguniform('regularization', 1e-5, 1e-3)


  0%|          | 0/144 [00:00<?, ?it/s]

[I 2023-04-30 08:47:19,593] Trial 0 finished with value: 0.1803598776459694 and parameters: {'factors': 58, 'regularization': 0.000752505453169848, 'iterations': 144}. Best is trial 0 with value: 0.1803598776459694.


  0%|          | 0/53 [00:00<?, ?it/s]

[I 2023-04-30 08:50:36,045] Trial 1 finished with value: 0.19547346606850624 and parameters: {'factors': 67, 'regularization': 0.00011399918721941122, 'iterations': 53}. Best is trial 1 with value: 0.19547346606850624.


...

  0%|          | 0/152 [00:00<?, ?it/s]

[I 2023-04-30 10:23:15,236] Trial 8 finished with value: 0.4351152032613754 and parameters: {'factors': 110, 'regularization': 2.4617339831522953e-05, 'iterations': 152}. Best is trial 5 with value: 0.926663413643837.


  0%|          | 0/93 [00:00<?, ?it/s]

[I 2023-04-30 10:32:51,275] Trial 9 finished with value: 0.5844213664531708 and parameters: {'factors': 143, 'regularization': 0.0005017949383814999, 'iterations': 93}. Best is trial 5 with value: 0.926663413643837.


Best Parameters:  {'factors': 195, 'regularization': 3.7097309328656386e-05, 'iterations': 111}
Best Score:  0.926663413643837


In [274]:
# Рекомендации для списка
userids = np.arange(10) 
ids, scores = model_user.recommend(userids, sparse_matrix[userids], N=5, filter_already_liked_items=True)
ids, ids.shape

(array([[193331, 111837, 169194, 116823, 192565],
        [163708, 159495, 210229, 156861, 178225],
        [116241, 147796, 155996, 111055, 240830],
        [242145, 153469, 126489, 127335, 160547],
        [128816, 113707, 133324, 153245, 134147],
        [154248, 154356, 244077, 126965, 158697],
        [153970, 246143, 116438, 249076, 198751],
        [128278, 137702, 117525, 161649, 253817],
        [128183, 117475, 101228, 182350, 170871],
        [245736, 147798, 171543, 116468, 110619]], dtype=int32),
 (10, 5))

In [277]:
userids = np.arange(10)
group_recommendations = model_user.recommend(userids, sparse_matrix[userids], N=5, filter_already_liked_items=True)

In [278]:
# Вот тут нужно написать код который записывает данные в датафрейм
result = pd.DataFrame({'пользователи' : userids, 'рекомендации' : list(ids)})

In [247]:
result.head()

,пользователи,рекомендации
0,0,"[193331, 111837, 169194, 116823, 192565]"
1,1,"[163708, 159495, 210229, 156861, 178225]"
2,2,"[116241, 147796, 155996, 111055, 240830]"
3,3,"[242145, 153469, 126489, 127335, 160547]"
4,4,"[128816, 113707, 133324, 153245, 134147]"


In [248]:
# Функция ранжирования 
def get_ranked_items(user_id, item_data, selected_items):
    return model_user.rank_items(user_id, item_data, selected_items=selected_items)


In [279]:
userid = 10005
selected_items = [1001, 1002, 10003, 100004]
rank_elements = get_ranked_items(userid, sparse_matrix, selected_items)
print(rank_elements)


(array([ 10003,   1002,   1001, 100004]), array([ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, -4.9881965e-06],
      dtype=float32))


/var/folders/9g/7jkvb2ws38ddzy34llt9htww0000gn/T/ipykernel_28542/874395100.py:3: DeprecationWarning: rank_items is deprecated. Use recommend with the 'items' parameter instead
  return model_user.rank_items(user_id, item_data, selected_items=selected_items)


In [250]:
test_df = pd.read_parquet('test_public_mfti.parquet', engine='pyarrow')
test_df

,cookie_id,vacancy_id_
0,000cd76cd33f43d4a1ac1d16d10f8bf7,"[222177, 222173, 222163, 238874, 238878, 22812..."
1,0034bc7f404341ba8412665453e7825a,"[102794, 137587, 257319, 237756, 240744, 11348..."
2,00a6c5a64a274c55a836402bdeb3b2c4,"[254292, 164602, 116438, 228634, 218819, 24065..."
3,015937a125b14e74bdff1cddc49f9172,"[246685, 138123, 115420, 210628, 212325, 235196]"
4,01de50c280794cec8804f16f45f847b7,"[219070, 251469, 166899, 212703, 214561]"
...,...,...
767,fdbcda17f22f406486837059e76c7fed,"[207851, 254989, 213344, 214180, 222146]"
768,fe6193ab26494ace9be5aae36e507618,"[115352, 230546, 225527, 120188, 109360, 23212..."
769,fe95b2826ee1452b81201ed3f4c3294d,"[240362, 114852, 253946, 251081, 127546, 244688]"
770,ff1aef256a49481698bb2e938510ff36,"[231194, 236363, 220747, 244688, 100094, 24052..."


In [251]:
# Сопоставляем номерные идексы с куки
test_df['numeric'] = test_df['cookie_id'].map(user_id_dict)

In [252]:
test_df

,cookie_id,vacancy_id_,numeric
0,000cd76cd33f43d4a1ac1d16d10f8bf7,"[222177, 222173, 222163, 238874, 238878, 22812...",54
1,0034bc7f404341ba8412665453e7825a,"[102794, 137587, 257319, 237756, 240744, 11348...",266
2,00a6c5a64a274c55a836402bdeb3b2c4,"[254292, 164602, 116438, 228634, 218819, 24065...",810
3,015937a125b14e74bdff1cddc49f9172,"[246685, 138123, 115420, 210628, 212325, 235196]",1727
4,01de50c280794cec8804f16f45f847b7,"[219070, 251469, 166899, 212703, 214561]",2395
...,...,...,...
767,fdbcda17f22f406486837059e76c7fed,"[207851, 254989, 213344, 214180, 222146]",327214
768,fe6193ab26494ace9be5aae36e507618,"[115352, 230546, 225527, 120188, 109360, 23212...",328026
769,fe95b2826ee1452b81201ed3f4c3294d,"[240362, 114852, 253946, 251081, 127546, 244688]",328303
770,ff1aef256a49481698bb2e938510ff36,"[231194, 236363, 220747, 244688, 100094, 24052...",329005


In [253]:
# Получаем список пользователей для загрузки в модель
user_ids = list(test_df['numeric'])
user_ids_str = list(test_df['cookie_id'])

# Получаем рекомендации для пользователей в списке
item_ids, scores = model_user.recommend(user_ids, sparse_matrix[user_ids], N=5, filter_already_liked_items=True)

# Создаем DataFrame с рекомендациями
recommendations_df = pd.DataFrame({'cookie_id': user_ids_str, 'top_recommendations': list(item_ids)})

In [254]:
recommendations_df

,cookie_id,top_recommendations
0,000cd76cd33f43d4a1ac1d16d10f8bf7,"[146630, 215620, 156844, 171415, 164698]"
1,0034bc7f404341ba8412665453e7825a,"[254741, 181275, 125869, 254643, 221349]"
2,00a6c5a64a274c55a836402bdeb3b2c4,"[109079, 227708, 113305, 231500, 225096]"
3,015937a125b14e74bdff1cddc49f9172,"[192565, 114583, 193331, 176171, 209568]"
4,01de50c280794cec8804f16f45f847b7,"[104528, 252155, 256268, 122122, 241827]"
...,...,...
767,fdbcda17f22f406486837059e76c7fed,"[164481, 168935, 202663, 170302, 128183]"
768,fe6193ab26494ace9be5aae36e507618,"[148686, 150516, 227708, 114918, 246680]"
769,fe95b2826ee1452b81201ed3f4c3294d,"[246815, 108201, 159041, 109959, 100898]"
770,ff1aef256a49481698bb2e938510ff36,"[163708, 209840, 106676, 178225, 159495]"


In [255]:
def precision_n(predictions_df, test_df_top, k=5):
    # Вычисляем общее количество рекомендаций для всех пользователей
    total_recommendations = len(test_df_top) * k

    # Вычисляем количество рекомендаций, которые были на самом деле интересны пользователю
    num_matches = 0
    for row in range(len(predictions_df)):
        predicted_items = set(predictions_df['top_recommendations'][row])
        true_items = set(test_df_top['vacancy_id_'][row])
        row_matches = predicted_items.intersection(true_items)
        num_matches += len(row_matches)

    # Вычисляем точность рекомендаций
    precision = num_matches / total_recommendations if total_recommendations != 0 else 0
    return precision

In [256]:
precision_n(recommendations_df, test_df)

0.04481865284974093

In [266]:
private_test_df = pd.read_parquet('test_private_users_mfti.parquet')


In [267]:
# Сопоставляем номерные идексы с куки
private_test_df['numeric'] = private_test_df['cookie_id'].map(user_id_dict)
private_test_df

,cookie_id,numeric
0,0018914ba3e54011b28fa715583d3354,109
1,0035c298d8c64f368ae730a9cca9bb20,275
2,00956458877448ec9fba87fb97443fdf,722
3,0099387c921b41e7bae6c99dd8254b60,739
4,009f65e8ae99413a8da94a491320580a,766
...,...,...
3081,ffadd195859444d2ade2479b0611c5c1,329779
3082,ffbc08b528c64f22996873fc63872202,329841
3083,ffdeaf3c34544529880aebf17c103f6c,330021
3084,ffefa79a74804ee69e6c131e0d05b948,330110


In [268]:
# Получаем список пользователей для загрузки в модель
user_ids = list(private_test_df['numeric'])
user_ids_str = list(private_test_df['cookie_id'])

# Получаем рекомендации для пользователей в списке
item_ids, scores = model_user.recommend(user_ids, sparse_matrix[user_ids], N=5, filter_already_liked_items=True)

# Создаем DataFrame с рекомендациями
private_recommendations_df = pd.DataFrame({'cookie_id': user_ids_str, 'top_recommendations': list(item_ids)})

In [271]:
private_recommendations_df.to_parquet('test_private_sample_submission_mfti_ALS3.parquet', index=False)
pd.read_parquet('test_private_sample_submission_mfti_ALS3.parquet')

,cookie_id,top_recommendations
0,0018914ba3e54011b28fa715583d3354,"[182382, 155615, 154656, 111848, 253817]"
1,0035c298d8c64f368ae730a9cca9bb20,"[135430, 254514, 163445, 253549, 236774]"
2,00956458877448ec9fba87fb97443fdf,"[154356, 126965, 101462, 119161, 119095]"
3,0099387c921b41e7bae6c99dd8254b60,"[173337, 202969, 125869, 173015, 204473]"
4,009f65e8ae99413a8da94a491320580a,"[120188, 164588, 121955, 102693, 252106]"
...,...,...
3081,ffadd195859444d2ade2479b0611c5c1,"[106943, 103881, 153970, 190928, 173337]"
3082,ffbc08b528c64f22996873fc63872202,"[206907, 189317, 198095, 214245, 112819]"
3083,ffdeaf3c34544529880aebf17c103f6c,"[181976, 257631, 178208, 213822, 200472]"
3084,ffefa79a74804ee69e6c131e0d05b948,"[220718, 146630, 207423, 237288, 156844]"
